In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.svm import SVC
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report
from imblearn.over_sampling import SMOTE

In [2]:
#code found from http://www.davidsbatista.net/blog/2018/02/23/model_optimization/
class EstimatorSelectionHelper:

    def __init__(self, models, params):
        if not set(models.keys()).issubset(set(params.keys())):
            missing_params = list(set(models.keys()) - set(params.keys()))
            raise ValueError("Some estimators are missing parameters: %s" % missing_params)
        self.models = models
        self.params = params
        self.keys = models.keys()
        self.grid_searches = {}

    def fit(self, X, y, cv=3, n_jobs=-1, verbose=1, scoring=None, refit=False):
        for key in self.keys:
            print("Running GridSearchCV for %s." % key)
            model = self.models[key]
            params = self.params[key]
            gs = GridSearchCV(model, params, cv=cv, n_jobs=n_jobs,
                              verbose=verbose, scoring=scoring, refit=refit,
                              return_train_score=True)
            gs.fit(X,y)
            self.grid_searches[key] = gs    

    def score_summary(self, sort_by='mean_score'):
        def row(key, scores, params):
            d = {
                 'estimator': key,
                 'min_score': min(scores),
                 'max_score': max(scores),
                 'mean_score': np.mean(scores),
                 'std_score': np.std(scores),
            }
            return pd.Series({**params,**d})

        rows = []
        for k in self.grid_searches:
            print(k)
            params = self.grid_searches[k].cv_results_['params']
            scores = []
            for i in range(self.grid_searches[k].cv):
                key = "split{}_test_score".format(i)
                r = self.grid_searches[k].cv_results_[key]        
                scores.append(r.reshape(len(params),1))

            all_scores = np.hstack(scores)
            for p, s in zip(params,all_scores):
                rows.append((row(k, s, p)))

        df = pd.concat(rows, axis=1).T.sort_values([sort_by], ascending=False)

        columns = ['estimator', 'min_score', 'mean_score', 'max_score', 'std_score']
        columns = columns + [c for c in df.columns if c not in columns]

        return df[columns]


In [3]:
classifier_model_params = {
    'LogisticRegression' : {
        'penalty' : ['l1', 'l2'],
        'C' : np.arange(.05, 1, .05) },
    'KNN' : {
        'n_neighbors' : np.arange(3, 22, 2) },
    'NaiveBayes' : {
        'alpha' : np.arange(.05, 2, .05)},
    'DecisionTree': {
        'max_depth' : [None, 6, 10, 14], 
        'min_samples_leaf' : [1, 2],
        'min_samples_split': [2, 3] },
    'BaggedDecisionTree' : {
        'n_estimators' : [20, 60, 100] },
    'RandomForest' : {
        'n_estimators' : [20, 60, 100],
        'max_depth' : [None, 2, 6, 10],
        'min_samples_split' : [2, 3, 4] },
    'ExtraTrees' : {
        'n_estimators' : [20, 60, 100],
        'max_depth' : [None, 6, 10, 14],
        'min_samples_leaf' : [1, 2], 
        'min_samples_split' : [2, 3], },
    'AdaBoost' : {
        'n_estimators' : np.arange(100, 151, 25),
        'learning_rate' : np.linspace(0.05, 1, 10) },
    'GradientBoosting' : {
        'n_estimators' : np.arange(5, 150, 15),
        'learning_rate' : np.linspace(0.05, 1, 10),
        'max_depth' : [1, 2, 3] },
    'SVM' : {
        'C' : np.arange(0.05, 1, .05),
        'kernel' : ['rbf', 'linear'] },
     'XGBoost' : {
        'n_estimators'  : np.arange(100, 151, 25), 
        'learning_rate' : np.arange(0.1, 1, .3),
        'max_depth' : [3],
        'alpha' : np.arange(0, 1, .3),
        'lambda' : np.arange(0, 1, .3),
        'gamma' : np.arange(0, 1, .3),
        'subsample' : [.5],
        'n_jobs' : [4],}
        }

In [4]:
scaler = StandardScaler()
sm = SMOTE(sampling_strategy=1,random_state=666)

train = pd.read_csv('./data/train_weather.csv')
train_dummies = pd.get_dummies(train,drop_first=True,columns=['Species','Street'])
y = train_dummies['WnvPresent']
X = train_dummies[[col for col in train_dummies.columns if col != 'WnvPresent']]

train_x, test_x, train_y, test_y = train_test_split(X,y,test_size = 0.3, random_state = 666)
sampledX,sampledy = sm.fit_sample(train_x,train_y)
sampledX=scaler.fit_transform(sampledX)
test_x=scaler.transform(test_x)

C:\Users\chang\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: DataConversionWarning: Data with input dtype uint8, int64, float64 were all converted to float64 by StandardScaler.
  if sys.path[0] == '':


In [14]:
classifier_models = {
#     'LogisticRegression' : /LogisticRegression(random_state = 42),
#     'KNN': KNeighborsClassifier(), 
# #     'NaiveBayes' : MultinomialNB(), #does not work with negative vals
#     'DecisionTree' : DecisionTreeClassifier(random_state = 42), 
#     'BaggedDecisionTree' : BaggingClassifier(random_state = 42),
#     'RandomForest' : RandomForestClassifier(random_state = 42), 
#     'ExtraTrees' : ExtraTreesClassifier(random_state = 42), 
#     'AdaBoost' : AdaBoostClassifier(random_state=42), 
    'GradientBoosting' : GradientBoostingClassifier(random_state = 42),
#     'SVM' : SVC(random_state=42),
#     'XGBoost' : XGBClassifier(random_state=42)
}

In [15]:
search = EstimatorSelectionHelper(classifier_models, classifier_model_params)
search.fit(sampledX,sampledy, scoring='recall', n_jobs=3)

Running GridSearchCV for GradientBoosting.
Fitting 3 folds for each of 300 candidates, totalling 900 fits


[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done  44 tasks      | elapsed:   30.0s
[Parallel(n_jobs=3)]: Done 194 tasks      | elapsed:  4.0min
[Parallel(n_jobs=3)]: Done 444 tasks      | elapsed:  9.6min
[Parallel(n_jobs=3)]: Done 794 tasks      | elapsed: 17.1min
[Parallel(n_jobs=3)]: Done 900 out of 900 | elapsed: 19.7min finished


We then score our different models and output our results to a csv for archival.

In [16]:
score2=search.score_summary(sort_by='max_score')

GradientBoosting


In [10]:
score1=pd.read_csv(r'.\data\score2.csv')

In [17]:
score1=pd.concat([score1,score2])

C:\Users\chang\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [18]:
score1.to_csv(r'.\data\score3.csv',index=False)

In [20]:
score1[score1['mean_score'] == score1.groupby('estimator')['mean_score'].transform('max')]

,C,alpha,estimator,gamma,kernel,lambda,learning_rate,max_depth,max_score,mean_score,min_samples_leaf,min_samples_split,min_score,n_estimators,n_jobs,n_neighbors,penalty,std_score,subsample
13,NaN,NaN,ExtraTrees,NaN,NaN,NaN,NaN,NaN,0.991387,0.984927,2.0,2.0,0.972868,60,NaN,NaN,NaN,0.00853394,NaN
14,NaN,NaN,BaggedDecisionTree,NaN,NaN,NaN,NaN,NaN,0.991387,0.957221,NaN,NaN,0.894057,100,NaN,NaN,NaN,0.0447135,NaN
15,NaN,NaN,ExtraTrees,NaN,NaN,NaN,NaN,NaN,0.991387,0.984927,2.0,3.0,0.972868,60,NaN,NaN,NaN,0.00853394,NaN
25,NaN,NaN,RandomForest,NaN,NaN,NaN,NaN,10,0.986219,0.971433,NaN,3.0,0.945306,100,NaN,NaN,NaN,0.018528,NaN
28,NaN,NaN,DecisionTree,NaN,NaN,NaN,NaN,14,0.985788,0.959231,1.0,2.0,0.921189,NaN,NaN,NaN,NaN,0.0275946,NaN
64,NaN,NaN,KNN,NaN,NaN,NaN,NaN,NaN,0.955642,0.942578,NaN,NaN,0.920758,NaN,NaN,3.0,NaN,0.0155291,NaN
135,0.15,NaN,LogisticRegression,NaN,NaN,NaN,NaN,NaN,0.860465,0.855154,NaN,NaN,0.851852,NaN,NaN,NaN,l2,0.00379266,NaN
142,0.05,NaN,LogisticRegression,NaN,NaN,NaN,NaN,NaN,0.860034,0.855154,NaN,NaN,0.850991,NaN,NaN,NaN,l2,0.00372689,NaN
153,NaN,NaN,AdaBoost,NaN,NaN,NaN,0.788889,NaN,0.964255,0.931668,NaN,NaN,0.867356,150,NaN,NaN,NaN,0.0454771,NaN
172,0.90,NaN,SVM,NaN,rbf,NaN,NaN,NaN,0.94186,0.923629,NaN,NaN,0.890612,NaN,NaN,NaN,NaN,0.0233892,NaN
